In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options  #options 目的是要去設定一些選項
from bs4 import BeautifulSoup as bs
import time
import os 
import pandas as pd
from tqdm import tqdm #progress bar 進度條
import warnings 
warnings.filterwarnings('ignore')
from datetime import datetime as dt
from datetime import date,timedelta
import csv

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
#設定
options = Options()
options.add_argument('--headless')    
options.add_argument('--disable-gpu') 

path = os.getcwd() 
driver = webdriver.Chrome(path+'/chromedriver',chrome_options=options)

In [11]:
#爬蟲
today = dt.today()
file_date = today.strftime('%m-%d')


#取得最新頁碼
url = 'https://www.ptt.cc/bbs/fastfood/index.html'
driver.get(url)
sourceCode = bs(driver.page_source,'html.parser')                                                     
button = sourceCode.select('a.btn.wide')[1]['href'] 
index = button.find('x')
dot = button.find('.')
page_num = button[index+1:dot]
print('lastest page:',int(page_num)+1)

try:
    with open (file_date+'PTT_fastfood.csv','w',newline='',encoding='utf_8_sig')as csvfile:
        fieldnames =['uni_index','title','author','year','date','url_article','push_positive','push_negative','content','p_c']
        writer = csv.writer(csvfile) 
        writer.writerow(fieldnames)  #把header寫進去 
           
        with open (file_date+'PTT_fastfood_comments.csv','w',newline='',encoding='utf_8_sig')as csvfile1:           
            fieldnames_push =['uni_index','year','date','push','push_author','comment']
            writer1 = csv.writer(csvfile1) 
            writer1.writerow(fieldnames_push)  #把header寫進去
                
            for i in tqdm(range(int(page_num)+1,1257,-1)): 
                url = 'https://www.ptt.cc/bbs/fastfood/index'+str(i)+'.html'
                print(i,url)
                driver.get(url)
                sourceCode = bs(driver.page_source,'html.parser')  
                metaSection = sourceCode.select('div.r-list-container')[0]
                sections = metaSection.select('div.r-ent')
                for section in sections :
                    try:
                        title = section.select('div.title')[0].text.strip()          
                        if (title.startswith("[公告]")): #公告文不抓
                            continue                                              
                        date=section.select('div.date')[0].text.strip()          
                        author=section.select('div.author')[0].text.strip()                    
                        href = section.select('.title')[0]('a')[0]['href']
                        index = href.split('.')[1] #文章key值
                        url_article = 'https://www.ptt.cc'+href
                        driver.get(url_article)
                        sourceCode = bs(driver.page_source,'html.parser') 
                        metaSection_year = sourceCode.select('span.article-meta-value')
                        year = metaSection_year[3].text.split()[-1] #取發文年份                   
                        metaSection_a = sourceCode.select('div#main-content') #取內文
                        content = metaSection_a[0].text.split(year)[1].split("※")[0].strip()
                        push_symbol = metaSection_a[0].select('span.hl.push-tag') #取推噓次數
                        push_positive = 0
                        push_negative = 0
                        for s in push_symbol:
                            symbol = s.text.strip()
                            if symbol == '推':
                                push_positive += 1
                            elif symbol == '噓':
                                push_negative += 1
                        push_content = metaSection_a[0].select('span.f3.push-content') #取推文
                        p_c = ""
                        for p in push_content:
                            push = p.text.split(':')[1].strip()
                            p_c += push+','

                         #把資料寫進去) 
                        writer.writerow([index,title,author,year,date,url_article,push_positive,push_negative,content,p_c]) 


                                               
                         #只存推文       
                        push = metaSection_a[0].select('div.push')
                        for p in push :
                            symbol = p.select('span.hl.push-tag')[0].text.strip()
                            push_content = p.select('span.f3.push-content')[0].text.split(':')[1].strip()
                            push_author = p.select('span.f3.hl.push-userid')[0].text.strip()
                            writer1.writerow([index,year,date,symbol,push_author,push_content])    

                    except Exception as e: 
                        print('error',",",e)
                        pass
                        
    driver.close() 
    
except Exception as e:
    print(e)
    driver.close()
    print('drive closed')


  0%|                                                                                           | 0/18 [00:00<?, ?it/s]

lastest page: 1275
1275 https://www.ptt.cc/bbs/fastfood/index1275.html
error , list index out of range



  6%|████▌                                                                              | 1/18 [00:17<05:04, 17.92s/it]

error , list index out of range
1274 https://www.ptt.cc/bbs/fastfood/index1274.html



 11%|█████████▏                                                                         | 2/18 [01:03<06:58, 26.13s/it]

...


 89%|████████████████████████████████████████████████████████████████████████▉         | 16/18 [14:19<01:31, 45.54s/it]

1259 https://www.ptt.cc/bbs/fastfood/index1259.html



 94%|█████████████████████████████████████████████████████████████████████████████▍    | 17/18 [14:56<00:42, 42.81s/it]

1258 https://www.ptt.cc/bbs/fastfood/index1258.html



100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [15:50<00:00, 52.78s/it]


In [ ]:
#資料清洗

In [28]:
df =pd.read_csv('04-20PTT_fastfood.csv',encoding='utf-8')
df_comments =pd.read_csv('04-20PTT_fastfood_comments.csv',encoding='utf-8')
df_calender =pd.read_csv('calender.csv',encoding='utf-8',parse_dates=['Date'])
print(df.info())
print(df_comments.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339 entries, 0 to 338
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   uni_index      339 non-null    int64 
 1   title          339 non-null    object
 2   author         339 non-null    object
 3   year           339 non-null    int64 
 4   date           339 non-null    object
 5   url_article    339 non-null    object
 6   push_positive  339 non-null    int64 
 7   push_negative  339 non-null    int64 
 8   content        316 non-null    object
 9   p_c            335 non-null    object
dtypes: int64(4), object(6)
memory usage: 26.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13537 entries, 0 to 13536
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   uni_index    13537 non-null  int64 
 1   year         13537 non-null  int64 
 2   date         13537 non-null  object
 3   push 

In [59]:
# 資料清洗

#文章清洗
def clean_ptt (df):
    #轉換type
    convert_dict = {'push_positive': int, 
                    'push_negative': int,
                    'uni_index':str,
                    'year':str
                   } 
  
    df = df.astype(convert_dict)  
    
    
    # 用推文+噓文計算總回復數
    df['all_num']=df['push_positive']+df['push_negative']
    
    # null
    df = df.fillna("")
    
    # 品牌分類 #esc+f全部取代
    # 以title判斷品牌,title沒有的再以content判斷 
    # 後面會取代前面,所以先以content判斷,如果title有關鍵字會取代content
    df.loc[df["content"].str.contains("必勝客"),"brand"]="必勝客"
    df.loc[df["content"].str.contains("麥當勞|小麥|麥噹噹|麥|點點卡"),"brand"]="麥當勞"
    df.loc[df["content"].str.contains("漢堡王"),"brand"]="漢堡王"
    df.loc[df["content"].str.contains("肯德基|KFC|kfc"),"brand"]="肯德基"
    df.loc[df["content"].str.contains("摩斯|mos|MOS"),"brand"]="摩斯漢堡"
    df.loc[df["content"].str.contains("21世紀"),"brand"]="21世紀"
    df.loc[df["content"].str.contains("丹丹"),"brand"]="丹丹"
    df.loc[df["content"].str.contains("必勝客"),"brand"]="必勝客"
    df.loc[df["content"].str.contains("達美樂"),"brand"]="達美樂"
    df.loc[df["content"].str.contains("subway"),"brand"]="subway"

    df.loc[df["title"].str.contains("麥當勞|小麥|麥噹噹|麥|點點卡"),"brand"]="麥當勞"
    df.loc[df["title"].str.contains("漢堡王"),"brand"]="漢堡王"
    df.loc[df["title"].str.contains("肯德基|KFC|kfc"),"brand"]="肯德基"
    df.loc[df["title"].str.contains("摩斯|mos|MOS"),"brand"]="摩斯漢堡"
    df.loc[df["title"].str.contains("21世紀"),"brand"]="21世紀"
    df.loc[df["title"].str.contains("丹丹"),"brand"]="丹丹"
    df.loc[df["title"].str.contains("達美樂"),"brand"]="達美樂"
    df.loc[df["title"].str.contains("subway"),"brand"]="subway"
    df.loc[df["title"].str.contains("必勝客"),"brand"]="必勝客"
    df.loc[df["title"].str.contains("拿坡里"),"brand"]="拿坡里"
    df.loc[df["brand"].isnull(),"brand"]="其他"
    
    # 標題切割成 標題跟文章分類 
    # 部分文章沒有選分類，所以比對不到分類,就直接把資料放入title
    # 如果不符合，則文章分類一律設定成"討論"，標題保持原樣
    category = ["問題","商品","討論","食記","情報","新聞","抱怨","推薦","公告","閒聊","問卷","分享","心得","政見","提醒"]
    cate_ ,title_ = [],[] 

    for a in df["title"].str.split("]"):
        ca = a[0].replace("[","").replace("Fw:","").replace("Re:","").strip()  

        if ca not in category :
            cate_.append("討論")
            title_.append(a[0])
        else:
            t = a[1]
            cate_.append(ca)
            title_.append(t)

    df["title"] = title_
    df["category"] = cate_  
   
 
    #轉換日期
    df['date'] = df['year']+"/"+df['date']
    df['date'] = pd.to_datetime(df['date'],errors="coerce")
    
    #drop year date num
    df.drop('year', axis=1,inplace=True)
    
    return df 



#推文清洗
def clean_push(df,df_push):
       
    #轉換type
    convert_dict = {'uni_index':str,
                    'year':str,
                    'comment':str}
  
    df_push = df_push.astype(convert_dict)  
    df_push = pd.merge(df_push,df[['uni_index','brand','category','title']],on='uni_index',how='left')
    df_push['date'] = df_push['year']+"/"+df_push['date']
    df_push['date'] = pd.to_datetime(df_push['date'],errors="coerce")
    df_push["comment"] = df_push["comment"].replace("",'空白留言') #因snowNLP如有空白資料會出現error
    df_push.drop('year', axis=1,inplace=True)
    
    #把相同文章相同推文者，歸類回同一欄
    df_push3 = df_push[["uni_index",'push_author','comment']]
    df_push_uni_com = df_push3.groupby(['uni_index','push_author'])["comment"].apply(lambda x:x.sum()).reset_index()   
    df_push_uni_article = df_push[["uni_index",'date','brand','category','title']].drop_duplicates('uni_index',keep='first')   
    df_push_author = df_push[['uni_index','push_author','push']].drop_duplicates(['uni_index','push_author'],keep='first')  
    df_push6 = pd.merge(df_push_uni_com, df_push_uni_article,on='uni_index',how='left')
    
    df_push_author['id_author']=df_push_author['uni_index']+'=='+df_push_author['push_author']
    df_push6['id_author']=df_push6['uni_index']+'=='+df_push6['push_author']
    df_push_all = pd.merge(df_push6,df_push_author[['id_author','push']],on='id_author',how='left')
    df_push_all.drop('id_author', axis=1,inplace=True)
    
    
    return df_push_all

In [62]:
df_clean = clean_ptt(df)
df_push2 = clean_push(df_clean,df_comments)
print(df_clean.info())
print(df_push2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339 entries, 0 to 338
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   uni_index      339 non-null    object        
 1   title          339 non-null    object        
 2   author         339 non-null    object        
 3   date           339 non-null    datetime64[ns]
 4   url_article    339 non-null    object        
 5   push_positive  339 non-null    int32         
 6   push_negative  339 non-null    int32         
 7   content        339 non-null    object        
 8   p_c            339 non-null    object        
 9   all_num        339 non-null    int32         
 10  brand          339 non-null    object        
 11  category       339 non-null    object        
dtypes: datetime64[ns](1), int32(3), object(8)
memory usage: 27.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9914 entries, 0 to 9913
Data columns (total 8 columns):


In [63]:
#建立維度表
#brand
df_brand = df_clean.drop_duplicates(subset='brand',keep='first')['brand'].reset_index(drop=True)
#category
df_category = df_clean.drop_duplicates(subset='category',keep='first')['category'].reset_index(drop=True)

In [ ]:
#情緒分析snowNLP

In [66]:
#情緒分析snownlp(pip install snownlp)
from snownlp import seg
from snownlp import SnowNLP

brand_list = df_push2['brand'].unique().tolist()
nlp = []
for i,b in enumerate(brand_list):
    print(i,b,'===================================================')
    brand_posts=df_push2[df_push2["brand"]==b]['comment']
    try:    
        for p in brand_posts:
            s = SnowNLP(p)
            sentiments = s.sentiments
            nlp.append(sentiments)
    except Exception as e:
        print('error:', e)
        pass
    
df_push2["sentiment"] = nlp
df_push2["sentiment_score"] = df_push2["sentiment"].map(lambda x :"正面" if x>=0.6 else "中立" if x>=0.5  else "負面" )
df_push2[0:600]

0 麥當勞 ===================================================
1 摩斯漢堡 ===================================================
2 必勝客 ===================================================
3 肯德基 ===================================================
4 達美樂 ===================================================
5 其他 ===================================================
6 漢堡王 ===================================================
7 丹丹 ===================================================
8 拿坡里 ===================================================
9 subway ===================================================


,uni_index,push_author,comment,date,brand,category,title,push,sentiment,sentiment_score
0,1577520990,S890127,因為她前面有先去櫃台求助一次 店家有警覺了吧,2019-12-28,麥當勞,新聞,神救援！受虐女子嘴型求救 加州麥當勞員,推,0.002174,負面
1,1577520990,amadda,謝謝他們的幫忙,2019-12-28,麥當勞,新聞,神救援！受虐女子嘴型求救 加州麥當勞員,推,0.371755,負面
2,1577520990,batis,太強了吧 原來得來速嘴型可以看這麼清楚?,2019-12-28,麥當勞,新聞,神救援！受虐女子嘴型求救 加州麥當勞員,推,0.435879,負面
3,1577520990,claudia1988,很棒,2019-12-28,麥當勞,新聞,神救援！受虐女子嘴型求救 加州麥當勞員,推,0.864865,正面
4,1577520990,ex250203,用氣音講吧 應該很明顯看的出來,2019-12-28,麥當勞,新聞,神救援！受虐女子嘴型求救 加州麥當勞員,→,0.184570,負面
...,...,...,...,...,...,...,...,...,...,...
595,1578435325,sashimi254,今年的黃金薯皮比較脆可是也漲價了,2020-01-08,摩斯漢堡,新聞,太「蝦」了！ 摩斯新品雙雙蝦堡豪邁包入4,推,0.470026,負面
596,1578435325,soxbox,可樂餅www,2020-01-08,摩斯漢堡,新聞,太「蝦」了！ 摩斯新品雙雙蝦堡豪邁包入4,→,0.113010,負面
597,1578435325,steak353,太貴了吧,2020-01-08,摩斯漢堡,新聞,太「蝦」了！ 摩斯新品雙雙蝦堡豪邁包入4,推,0.447453,負面
598,1578435325,urban01,可樂餅那個栗子多餘砍掉 賣40不行嗎,2020-01-08,摩斯漢堡,新聞,太「蝦」了！ 摩斯新品雙雙蝦堡豪邁包入4,推,0.962764,正面


In [73]:
#資料輸出
df_clean.to_csv('D:/ptt_clean.csv',encoding='utf-8',index=False)
df_push2.to_csv('D:/ptt_push_clean.csv',encoding='utf-8',index=False)
df_brand.to_csv('D:/ptt_brand.csv',encoding='utf-8',index=False)
df_category.to_csv('D:/ptt_category.csv',encoding='utf-8',index=False)

In [ ]:
#關鍵字分析 - 抓累計區間以及每週關鍵字

In [68]:
import jieba
import jieba.analyse
import os
from collections import Counter #計算list內同一個詞彙出現幾次

In [69]:
#merge周別(內文)
df_clean_w = pd.merge(df_clean,df_calender[['Date','月週']],left_on='date',right_on='Date',how='left')
df_clean_w.drop('Date',axis=1,inplace=True)

#merge周別(推文)
df_push2 = pd.merge(df_push2,df_calender[['Date','月週']],left_on='date',right_on='Date',how='left')
df_push2.drop('Date',axis=1,inplace=True)

print(df_clean_w.info())
print(df_push2.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 339 entries, 0 to 338
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   uni_index      339 non-null    object        
 1   title          339 non-null    object        
 2   author         339 non-null    object        
 3   date           339 non-null    datetime64[ns]
 4   url_article    339 non-null    object        
 5   push_positive  339 non-null    int32         
 6   push_negative  339 non-null    int32         
 7   content        339 non-null    object        
 8   p_c            339 non-null    object        
 9   all_num        339 non-null    int32         
 10  brand          339 non-null    object        
 11  category       339 non-null    object        
 12  月週             330 non-null    object        
dtypes: datetime64[ns](1), int32(3), object(9)
memory usage: 33.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9914 

In [70]:
# 1.匯入停用詞典
def stopwordslist(filepath):  
 stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
 return stopwords  
#==========================================================================================
# 2.匯入停用詞
path = 'stopwords.txt'
stopwords = stopwordslist(path)  
stopwords

['\ufeff,',
 '?',
 '、',
 '。',
 '“',
 '”',
 '《',
 '》',
 '！',
 '，',
 '：',
 '；',
 '？',
 '人民',
 '末##末',
 '啊',
 '阿',
 '哎',
 '哎呀',
 '哎喲',
 '唉',
 '我',
 '我們',
 '按',
 '按照',
 '依照',
 '吧',
 '吧噠',
 '把',
 '罷了',
 '被',
 '本',
 '本著',
 '比',
 '比方',
 '比如',
 '鄙人',
 '彼',
 '彼此',
 '邊',
 '別',
 '別的',
 '別說',
 '並',
 '並且',
 '不比',
 '不成',
 '不單',
 '不但',
 '不獨',
 '不管',
 '不光',
 '不過',
 '不僅',
 '不拘',
 '不論',
 '不怕',
 '不然',
 '不如',
 '不特',
 '不惟',
 '不問',
 '不只',
 '朝',
 '朝著',
 '趁',
 '趁著',
 '乘',
 '沖',
 '除',
 '除此之外',
 '除非',
 '除了',
 '此',
 '此間',
 '此外',
 '從',
 '從而',
 '打',
 '待',
 '但',
 '但是',
 '當',
 '當著',
 '到',
 '得',
 '的',
 '的話',
 '等',
 '等等',
 '地',
 '第',
 '叮咚',
 '對',
 '對於',
 '多',
 '多少',
 '而',
 '而況',
 '而且',
 '而是',
 '而外',
 '而言',
 '而已',
 '爾後',
 '反過來',
 '反過來說',
 '反之',
 '非但',
 '非徒',
 '否則',
 '嘎',
 '嘎登',
 '該',
 '趕',
 '個',
 '各',
 '各個',
 '各位',
 '各種',
 '各自',
 '給',
 '根據',
 '跟',
 '故',
 '故此',
 '固然',
 '關於',
 '管',
 '歸',
 '果然',
 '果真',
 '過',
 '哈',
 '哈哈',
 '呵',
 '和',
 '何',
 '何處',
 '何況',
 '何時',
 '嘿',
 '哼',
 '哼唷',
 '呼哧',
 '乎',
 '嘩',
 '還是',
 '還有',
 '換句話說',
 '

In [ ]:
#抓累計區間關鍵字

In [72]:
#抓累計區間內文關鍵字
today = dt.today()
id_today = today.strftime('%Y/%m/%d')


try:
    with open (file_date+'content_keywords.csv','w',newline='',encoding='utf_8_sig')as csvfile3:
        fieldnames_content =['id_date','brand','keywords','frequency']
        writer3 = csv.writer(csvfile3) 
        writer3.writerow(fieldnames_content)  #把header寫進去 
                
        brand_list = df_clean['brand'].unique().tolist()
        # 針對不同品牌的文章抓出最關鍵字擷取
        for i,b in enumerate(brand_list):
            print(i,b,'===================================================')
            brand_posts=df_clean[df_clean["brand"]==b]['content']
            time = 0

            re = []
            for p in brand_posts:
                time = time+1
                key_words = jieba.analyse.extract_tags(p,5)#list資料
                re = re+key_words
                 

            #比對如果為停用詞拿掉
            results_keywords = []
            for word in re :
                if word not in stopwords :       
                    results_keywords.append(word)

            #排序文字
            frequencies = Counter(results_keywords) #計算每個詞出現次數(list中)
            frequencies = frequencies.most_common() #依照出現次數排序
            frequencies = frequencies[:20]

            for f in frequencies:
                keywords =f[0].strip()
                freq = f[1]
                print('=====',b)
                print(keywords,freq)

                writer3.writerow([id_today,b,keywords,freq]) 
            
except Exception as e:
    print('error',",",e) 
    print('done')

0 麥當勞 ===================================================
===== 麥當勞
麥當勞 44
===== 麥當勞
漢堡 8
===== 麥當勞
豬肉 6
===== 麥當勞
店員 5
===== 麥當勞
長堡 5
===== 麥當勞
比較 5
===== 麥當勞
單點 5
===== 麥當勞
時間 4
===== 麥當勞
得來速 4
===== 麥當勞
雞肉 3
===== 麥當勞
加菜 3
===== 麥當勞
點餐 3
===== 麥當勞
小薯 3
===== 麥當勞
Fri 3
===== 麥當勞
廣告 3
===== 麥當勞
麥脆 3
===== 麥當勞
優惠券 3
===== 麥當勞
照燒 3
===== 麥當勞
滿福堡 3
===== 麥當勞
兌換 2
1 肯德基 ===================================================
===== 肯德基
蛋塔 11
===== 肯德基
肯德基 7
===== 肯德基
KFC 6
===== 肯德基
雞皮 6
===== 肯德基
炸雞 5
===== 肯德基
漢堡 4
===== 肯德基
優惠 4
===== 肯德基
焙茶 3
===== 肯德基
脆腿堡 3
===== 肯德基
花生 3
===== 肯德基
外送 3
===== 肯德基
雞腿堡 3
===== 肯德基
金沙 3
===== 肯德基
優惠券 2
===== 肯德基
早餐 2
===== 肯德基
速食 2
===== 肯德基
活動 2
===== 肯德基
花生醬 2
===== 肯德基
43 2
===== 肯德基
培根 2
2 必勝客 ===================================================
===== 必勝客
必勝客 8
===== 必勝客
披薩 8
===== 必勝客
比薩 5
===== 必勝客
三杯雞 4
===== 必勝客
干貝 2
===== 必勝客
金箔 2
===== 必勝客
抹茶 2
===== 必勝客
熊貓 2
===== 必勝客
活動 2
===== 必勝客
口味 2
===== 必勝客
起司 2
===== 必勝客
比較 2
===== 必勝客
海鮮 2
===== 必勝客
未來 1
=

In [74]:
#抓累計推文關鍵字
try:
    with open (file_date+'push_keywords.csv','w',newline='',encoding='utf_8_sig')as csvfile4:
        fieldnames_push =['id_date','brand','keywords','frequency']
        writer4 = csv.writer(csvfile4) 
        writer4.writerow(fieldnames_push)  #把header寫進去 
        
        brand_list = df_clean['brand'].unique().tolist()
                # 針對不同品牌的文章抓出最關鍵字擷取
        for i,b in enumerate(brand_list):
            print(i,b,'===================================================')
            brand_posts=df_clean[df_clean["brand"]==b]['p_c']
            time = 0

            res = []
            for p in brand_posts:
                time = time+1
                key_words = jieba.analyse.extract_tags(p,5)#list資料
                res = res+key_words

            #比對如果為停用詞拿掉
            results_keywords_push = []           
            for word in res :
                if word not in stopwords :       
                    results_keywords_push.append(word)

            #排序文字
            frequencies = Counter(results_keywords_push) #計算每個詞出現次數(list中)
            frequencies = frequencies.most_common() #依照出現次數排序
            frequencies = frequencies[:20]
            print(frequencies)

            for f in frequencies:
                keywords =f[0].strip()
                freq = f[1]
                print(keywords,freq)

                writer4.writerow([id_today,b,keywords,freq]) 
                

    print('already write to csv')
except Exception as e:
    print('error',",",e)
    print('done')

0 麥當勞 ===================================================
[('麥當勞', 16), ('漢堡', 12), ('好吃', 9), ('台灣', 7), ('比較', 7), ('店員', 6), ('優惠', 6), ('麥香', 6), ('甜心卡', 6), ('麵包', 5), ('難吃', 5), ('飲料', 5), ('點卡', 4), ('大麥克', 4), ('薯餅', 4), ('漢堡王', 3), ('豬肉', 3), ('蘋果派', 3), ('感謝', 3), ('培根', 3)]
麥當勞 16
漢堡 12
好吃 9
台灣 7
比較 7
店員 6
優惠 6
麥香 6
甜心卡 6
麵包 5
難吃 5
飲料 5
點卡 4
大麥克 4
薯餅 4
漢堡王 3
豬肉 3
蘋果派 3
感謝 3
培根 3
1 肯德基 ===================================================
[('好吃', 11), ('蛋塔', 10), ('比較', 8), ('炸雞', 7), ('肯德基', 7), ('優惠券', 6), ('雞皮', 6), ('店員', 5), ('地瓜球', 5), ('優惠', 5), ('KFC', 4), ('喜歡', 4), ('麥當勞', 4), ('感謝', 4), ('59', 4), ('難吃', 3), ('花生', 3), ('雞腿', 3), ('熱量', 3), ('吸管', 3)]
好吃 11
蛋塔 10
比較 8
炸雞 7
肯德基 7
優惠券 6
雞皮 6
店員 5
地瓜球 5
優惠 5
KFC 4
喜歡 4
麥當勞 4
感謝 4
59 4
難吃 3
花生 3
雞腿 3
熱量 3
吸管 3
2 必勝客 ===================================================
[('披薩', 5), ('必勝客', 5), ('榴槤', 4), ('好吃', 4), ('起司', 4), ('謝謝', 3), ('素肉', 2), ('業配', 2), ('抹茶', 2), ('點數', 2), ('喜歡', 2), ('比較', 2), ('三杯雞', 2), ('餅皮', 2),

In [ ]:
#抓每週關鍵字

In [75]:
#每週內文


try:
     with open (file_date+'week_content_keywords.csv','w',newline='',encoding='utf_8_sig')as csvfile5:
        fieldnames_push =['weeknum','brand','keywords','frequency']
        writer5 = csv.writer(csvfile5) 
        writer5.writerow(fieldnames_push)  #把header寫進去 
 

        brand_list = df_clean['brand'].unique().tolist()
        weeknum_list = df_clean_w['月週'].unique().tolist()
        
        #先抓出每一週
        for t,a in enumerate(weeknum_list):
            weeknum_posts=df_clean_w[df_clean_w["月週"]==a]
            
            # 針對不同品牌的文章抓出最關鍵字擷取
            for i,b in enumerate(brand_list):
                print(t,a,i,b,'===================================================')
                brand_posts=weeknum_posts[weeknum_posts["brand"]==b]['content']
                time = 0

                re = []
                for p in brand_posts:
                    time = time+1
                    key_words = jieba.analyse.extract_tags(p,5)#list資料
                    re = re+key_words


                #比對如果為停用詞拿掉
                results_keywords = []
                for word in re :
                    if word not in stopwords :       
                        results_keywords.append(word)

                #排序文字
                frequencies = Counter(results_keywords) #計算每個詞出現次數(list中)
                frequencies = frequencies.most_common() #依照出現次數排序
                frequencies = frequencies[:20]

                for f in frequencies:
                    keywords =f[0].strip()
                    freq = f[1]
                    print('=====',b)
                    print(keywords,freq)
                    
                    writer5.writerow([a,b,keywords,freq]) 
                     
            
except Exception as e:
    print('error',",",e)


0 202004_4 0 麥當勞 ===================================================
===== 麥當勞
麥當勞 2
===== 麥當勞
linepay 1
===== 麥當勞
粉絲團 1
===== 麥當勞
距離 1
===== 麥當勞
後面 1
===== 麥當勞
排隊 1
===== 麥當勞
有沒有 1
0 202004_4 1 肯德基 ===================================================
0 202004_4 2 必勝客 ===================================================
0 202004_4 3 其他 ===================================================
0 202004_4 4 摩斯漢堡 ===================================================
0 202004_4 5 漢堡王 ===================================================
0 202004_4 6 丹丹 ===================================================
0 202004_4 7 subway ===================================================
0 202004_4 8 達美樂 ===================================================
0 202004_4 9 拿坡里 ===================================================
1 202004_3 0 麥當勞 ===================================================
===== 麥當勞
麥當勞 2
===== 麥當勞
點餐 2
===== 麥當勞
店員 2
===== 麥當勞
豬肉 1
===== 麥當勞
雞肉 1
===== 麥當勞
美味 1
===== 麥當勞
加菜 1
===== 麥當勞
要為 1
=====

===== 麥當勞
麥當勞 4
===== 麥當勞
餐盤 2
===== 麥當勞
清潔 2
===== 麥當勞
咖啡杯 1
===== 麥當勞
玉米湯 1
===== 麥當勞
打開 1
===== 麥當勞
外送 1
===== 麥當勞
雞腿 1
===== 麥當勞
02 1
===== 麥當勞
打烊 1
===== 麥當勞
服務員 1
===== 麥當勞
乾淨 1
===== 麥當勞
肥宅 1
===== 麥當勞
員工 1
===== 麥當勞
點餐 1
===== 麥當勞
店員 1
===== 麥當勞
得來速 1
===== 麥當勞
顯示 1
===== 麥當勞
集點 1
===== 麥當勞
環保杯 1
8 202003_2 1 肯德基 ===================================================
===== 肯德基
肯德基 2
===== 肯德基
點餐機 1
===== 肯德基
自助 1
===== 肯德基
使用 1
===== 肯德基
店員 1
===== 肯德基
登錄 1
===== 肯德基
怪怪 1
===== 肯德基
培根 1
===== 肯德基
勁爆 1
===== 肯德基
米花 1
===== 肯德基
外送 1
===== 肯德基
狀況 1
===== 肯德基
蛋塔 1
===== 肯德基
整盒 1
===== 肯德基
43 1
===== 肯德基
花生 1
===== 肯德基
beauty321 1
===== 肯德基
PK 1
===== 肯德基
點數 1
===== 肯德基
雙倍 1
8 202003_2 2 必勝客 ===================================================
===== 必勝客
披薩 1
===== 必勝客
三杯雞 1
===== 必勝客
感覺 1
===== 必勝客
99 1
===== 必勝客
69 1
8 202003_2 3 其他 ===================================================
===== 其他
網誌 1
===== 其他
wind 1
===== 其他
suzukihiro 1
===== 其他
炸雞 1
===== 其他
信義店 1
===== 其他
五號 1
===== 其他

===== 肯德基
米花 1
===== 肯德基
漲價 1
===== 肯德基
XL 1
===== 肯德基
購買 1
===== 肯德基
客人 1
===== 肯德基
比較 1
===== 肯德基
告知 1
===== 肯德基
炸雞 1
===== 肯德基
JOLLIBEE 1
===== 肯德基
香港 1
===== 肯德基
肯德基 1
14 202001_5 2 必勝客 ===================================================
14 202001_5 3 其他 ===================================================
14 202001_5 4 摩斯漢堡 ===================================================
===== 摩斯漢堡
蛋糕 1
===== 摩斯漢堡
長崎 1
===== 摩斯漢堡
摩斯 1
===== 摩斯漢堡
請教 1
===== 摩斯漢堡
正統 1
14 202001_5 5 漢堡王 ===================================================
===== 漢堡王
56 1
===== 漢堡王
時間 1
===== 漢堡王
Tue 1
===== 漢堡王
Jan 1
===== 漢堡王
28 1
===== 漢堡王
QK1181c 1
14 202001_5 6 丹丹 ===================================================
14 202001_5 7 subway ===================================================
14 202001_5 8 達美樂 ===================================================
14 202001_5 9 拿坡里 ===================================================
15 202002_1 0 麥當勞 ===================================================
===== 麥當勞
麥當勞 3
===

In [76]:
#抓每週推文關鍵字
try:
    with open (file_date+'week_push_keywords.csv','w',newline='',encoding='utf_8_sig')as csvfile6:
        fieldnames_push =['weeknum','brand','keywords','frequency']
        writer6 = csv.writer(csvfile6) 
        writer6.writerow(fieldnames_push)  #把header寫進去 
        
        
        brand_list = df_clean['brand'].unique().tolist()
        weeknum_list = df_clean_w['月週'].unique().tolist()
       
         #先抓出每一週
        for t,a in enumerate(weeknum_list):
            weeknum_push=df_clean_w[df_clean_w["月週"]==a]
                  
            # 針對不同品牌的文章抓出最關鍵字擷取
            for i,b in enumerate(brand_list):
                print(a,i,b,'===================================================')
                brand_posts=weeknum_push[weeknum_push["brand"]==b]['p_c']
                time = 0

                res = []
                for p in brand_posts:
                    time = time+1
                    key_words = jieba.analyse.extract_tags(p,5)#list資料
                    res = res+key_words

                #比對如果為停用詞拿掉
                results_keywords_push = []           
                for word in res :
                    if word not in stopwords :       
                        results_keywords_push.append(word)

                #排序文字
                frequencies = Counter(results_keywords_push) #計算每個詞出現次數(list中)
                frequencies = frequencies.most_common() #依照出現次數排序
                frequencies = frequencies[:20]
                print(frequencies)

                for f in frequencies:
                    keywords =f[0].strip()
                    freq = f[1]
                    print(keywords,freq)

                    writer6.writerow([a,b,keywords,freq]) 
                    

except Exception as e:
    print('error',",",e)

202004_4 0 麥當勞 ===================================================
[('點卡', 1), ('回饋', 1), ('優惠', 1), ('點數', 1), ('現金', 1), ('插隊', 1), ('戴口罩', 1), ('距離', 1), ('餐點', 1), ('打電話', 1)]
點卡 1
回饋 1
優惠 1
點數 1
現金 1
插隊 1
戴口罩 1
距離 1
餐點 1
打電話 1
202004_4 1 肯德基 ===================================================
[]
202004_4 2 必勝客 ===================================================
[]
202004_4 3 其他 ===================================================
[]
202004_4 4 摩斯漢堡 ===================================================
[]
202004_4 5 漢堡王 ===================================================
[]
202004_4 6 丹丹 ===================================================
[]
202004_4 7 subway ===================================================
[]
202004_4 8 達美樂 ===================================================
[]
202004_4 9 拿坡里 ===================================================
[]
202004_3 0 麥當勞 ===================================================
[('激素', 1), ('麵包', 1), ('雨八', 1), ('id', 1), ('店員', 1), ('錄影', 1), ('

[]
202003_4 0 麥當勞 ===================================================
[('麥當勞', 3), ('點卡', 1), ('儲值', 1), ('回饋', 1), ('點數', 1), ('培根', 1), ('照燒', 1), ('豬肉堡', 1), ('利潤', 1), ('供不應求', 1), ('更新', 1), ('Intel', 1), ('AMD', 1), ('紙包', 1), ('漢堡', 1), ('大麥克', 1), ('盒裝', 1), ('比較', 1), ('雞塊', 1), ('價格', 1)]
麥當勞 3
點卡 1
儲值 1
回饋 1
點數 1
培根 1
照燒 1
豬肉堡 1
利潤 1
供不應求 1
更新 1
Intel 1
AMD 1
紙包 1
漢堡 1
大麥克 1
盒裝 1
比較 1
雞塊 1
價格 1
202003_4 1 肯德基 ===================================================
[]
202003_4 2 必勝客 ===================================================
[('起司', 2), ('燒肉', 1), ('泡菜', 1), ('章魚燒', 1), ('夏威夷', 1), ('海鮮', 1), ('比較', 1), ('明顯', 1), ('必勝客', 1), ('3.5', 1), ('ya', 1), ('樓上', 1), ('紅豆', 1), ('照片', 1)]
起司 2
燒肉 1
泡菜 1
章魚燒 1
夏威夷 1
海鮮 1
比較 1
明顯 1
必勝客 1
3.5 1
ya 1
樓上 1
紅豆 1
照片 1
202003_4 3 其他 ===================================================
[('店員', 1), ('奶昔', 1), ('比較', 1), ('飲料', 1), ('太貴', 1), ('餐廳', 1), ('越來越', 1)]
店員 1
奶昔 1
比較 1
飲料 1
太貴 1
餐廳 1
越來越 1
202003_4 4 摩斯漢堡 ========================

[('感謝', 2), ('吸管', 1), ('環保', 1), ('海龜', 1), ('智障', 1), ('飲料', 1), ('杏鮑菇', 1), ('可憐', 1), ('賣完', 1), ('味道', 1), ('難吃', 1), ('停售', 1), ('女店員', 1), ('漢堡', 1), ('滅火', 1), ('經典', 1), ('好吃', 1), ('脆腿', 1), ('帳號', 1), ('食記', 1)]
感謝 2
吸管 1
環保 1
海龜 1
智障 1
飲料 1
杏鮑菇 1
可憐 1
賣完 1
味道 1
難吃 1
停售 1
女店員 1
漢堡 1
滅火 1
經典 1
好吃 1
脆腿 1
帳號 1
食記 1
202002_4 2 必勝客 ===================================================
[]
202002_4 3 其他 ===================================================
[('炸雞', 2), ('蘋果汁', 1), ('肝炸', 1), ('雞胗', 1), ('推薦', 1), ('特利', 1), ('下下', 1), ('搬家', 1)]
炸雞 2
蘋果汁 1
肝炸 1
雞胗 1
推薦 1
特利 1
下下 1
搬家 1
202002_4 4 摩斯漢堡 ===================================================
[('ID', 1), ('正確', 1), ('豬排', 1), ('蘆筍', 1), ('感覺', 1), ('干貝', 1), ('吃到', 1)]
ID 1
正確 1
豬排 1
蘆筍 1
感覺 1
干貝 1
吃到 1
202002_4 5 漢堡王 ===================================================
[('華堡', 1), ('已經', 1), ('大餐', 1), ('重磅', 1)]
華堡 1
已經 1
大餐 1
重磅 1
202002_4 6 丹丹 ===================================================
[]
202002_4 7 subway ========

[('點卡', 2), ('麥當勞', 2), ('比較', 2), ('收到', 2), ('FP', 1), ('風向', 1), ('熊貓', 1), ('大潤發', 1), ('品項', 1), ('美式', 1), ('味道', 1), ('咖啡粉', 1), ('怪怪', 1), ('點餐機', 1), ('排隊', 1), ('說真的', 1), ('路易', 1), ('說不定', 1), ('長大', 1), ('便當', 1)]
點卡 2
麥當勞 2
比較 2
收到 2
FP 1
風向 1
熊貓 1
大潤發 1
品項 1
美式 1
味道 1
咖啡粉 1
怪怪 1
點餐機 1
排隊 1
說真的 1
路易 1
說不定 1
長大 1
便當 1
202001_3 1 肯德基 ===================================================
[('雞皮', 3), ('好吃', 2), ('比較', 2), ('營業', 1), ('除夕', 1), ('麥當勞', 1), ('打電話', 1), ('去問', 1), ('份數', 1), ('賣完', 1), ('限量', 1), ('新埔', 1), ('飢餓', 1), ('滅火', 1), ('鹹酥雞', 1), ('感謝', 1), ('59', 1), ('30', 1), ('肯德基', 1), ('吃完', 1)]
雞皮 3
好吃 2
比較 2
營業 1
除夕 1
麥當勞 1
打電話 1
去問 1
份數 1
賣完 1
限量 1
新埔 1
飢餓 1
滅火 1
鹹酥雞 1
感謝 1
59 1
30 1
肯德基 1
吃完 1
202001_3 2 必勝客 ===================================================
[('餅皮', 1), ('舊金山', 1), ('鬆厚', 1), ('感謝', 1), ('分享', 1), ('19431', 1), ('註明', 1), ('謝謝', 1), ('頭髮', 1), ('紙券', 1), ('手機', 1), ('圖檔', 1)]
餅皮 1
舊金山 1
鬆厚 1
感謝 1
分享 1
19431 1
註明 1
謝謝 1
頭髮 1
紙券 1
手機 1
圖檔 1
202